# 📌 코드 작성 목적

- 프로젝트를 수행함에 있어 train 데이터셋의 크기가 개인 로컬 컴퓨터에서 다루기에는 용량이 매우 큰 편
- "Quick Draw!" 의 목적은 csv파일의 "word" 컬럼(label)의 이미지를 그린 획의 정보가 있는 "drawing" 컬럼의 정보를 사용해 이미지를 딥러닝 모델에 학습시켜, 특정 "word"를 맞추는 것입니다.
- 따라서, 효율적인 데이터 및 메모리 사용을 위해 필요한 정보만을 추출해서 병렬적으로 처리하기 위해 전체 데이터셋에서 필요한 컬럼들만 추출하여 100개의 파일로 chunk한 파일을 만듭니다.
    - 각각의 chunk 파일은 train 데이터셋에 잇는 340종의 이미지에 대한 csv파일별로 랜덤하지만 균일하게 정보를 추출해 담도록 했습니다.
    - 이렇게 생성된 100개의 `.gz` 파일을 사용해서 차후 모델에 chunk 파일에서 정보를 불러와서 이미지를 시각화해 모델에 학습시키는 데이터를 제공하는 데이터 파이프라인을 구축할 때 사용합니다.

## STEP 1. 전처리 함수 정의

- 이 함수는 CSV 파일을 전처리하여 여러 개의 압축된 CSV 파일로 분할하는 작업을 수행합니다. `preprocess_file` 함수는 원본 CSV 파일의 경로, 분할된 파일을 저장할 경로, 분할할 개수, 라벨, 처리된 파일의 로그를 기록할 파일의 경로를 인자로 받습니다. 주요 작업은 다음과 같습니다:

    - 원본 CSV 파일에서 필요한 열(`drawing`, `key_id`)을 읽어 `pandas` 데이터프레임으로 변환합니다.
    - 지정된 라벨을 데이터프레임에 추가합니다.
    - `key_id`를 기반으로 데이터를 분할할 열(`cv`)을 생성하고, 이를 사용하여 데이터를 지정된 개수만큼 분할합니다.
    - 각 분할된 데이터를 별도의 압축 파일(`gzip`)로 저장합니다. 파일이 이미 존재하는 경우, 데이터를 해당 파일에 추가합니다.
    - 원본 CSV 파일은 처리 후 삭제됩니다.

In [3]:
from tqdm import tqdm  # tqdm 라이브러리를 임포트하여 진행 상황을 시각화
import pandas as pd  # pandas 라이브러리를 데이터 처리를 위해 임포트
import numpy as np  # numpy 라이브러리를 수치 계산을 위해 임포트
import os  # os 라이브러리를 운영 체제와 상호 작용을 위해 임포트
import zipfile
import gzip  # gzip 라이브러리를 압축 파일 처리를 위해 임포트

def preprocess_file(csv_file, raw_path, raw_shuffle_data_path, divide_shuffles, label, processed_files_log):
    try:
        # CSV 파일을 읽어서 데이터프레임으로 저장합니다.
        df = pd.read_csv(raw_path + csv_file, usecols=["drawing", "key_id"])

        # 라벨 열을 추가하고 지정된 "label" 값을 넣습니다.
        df["y"] = label

        # "key_id" 값을 사용하여 "cv" 열을 생성합니다. 이 열은 데이터를 분할하는 데 사용됩니다.
        df["cv"] = (df.key_id // 10000) % divide_shuffles

        for k in range(divide_shuffles):
            # 새로운 파일 이름을 생성하여 압축 파일로 저장합니다.
            filename = raw_shuffle_data_path + f"train_k{k}.csv.gz"
            
            # "cv" 열 값이 "k"와 일치하는 행만 선택하여 "chunk"에 저장합니다.
            chunk = df[df.cv == k]
            
            # "key_id"와 "cv" 열을 삭제하여 필요한 열만 남깁니다.
            chunk = chunk.drop(["key_id", "cv"], axis=1)

            if not os.path.exists(filename):
                # 파일이 존재하지 않으면 새로 생성하고 데이터를 압축하여 저장합니다.
                chunk.to_csv(filename, index=False, compression="gzip")
            else:
                with gzip.open(filename, "at") as f:
                    # 파일이 이미 있으면 gzip 모드에서 데이터를 추가로 저장합니다.
                    chunk.to_csv(f, header=False, index=False)

        os.remove(raw_path + csv_file)  # 원본 CSV 파일을 삭제합니다.

    except Exception as e:
        print(f"오류 발생: {csv_file} 처리 중 - {e}")  # 오류 발생 시 메시지 출력
        
    with open(processed_files_log, "a") as log:
        log.write(csv_file + "\n")  # 처리된 파일 로그에 기록합니다.

## STEP 2. 압축 해제 및 전처리

- 이 함수는 Quick Draw 데이터셋의 압축 파일을 처리하고, 원본 데이터를 전처리하는 과정을 담당합니다. 
    - 먼저, 압축 파일의 경로, 원본 데이터 디렉토리의 경로, 무작위로 섞인 데이터를 저장할 디렉토리의 경로, 데이터를 나눌 부분의 수, 그리고 처리된 파일의 로그를 저장할 파일의 경로를 정의합니다. 
    - 이후, 필요한 디렉토리가 없는 경우 생성하고, 로그 파일에서 이미 처리된 파일 목록을 읽어옵니다. 
    - 압축 파일 내의 `.csv` 파일들을 찾아 진행 상황을 시각화하며, 아직 처리되지 않은 파일들에 대해 압축을 해제하고 전처리 함수를 호출합니다. 
    - 각 파일 처리 후에는 진행 상황을 업데이트하고, 모든 파일이 처리되면 진행 상황 표시를 종료합니다. 
    - 이 과정은 데이터 전처리 파이프라인의 일부로, 데이터를 분석이나 학습에 적합한 형태로 만들기 위함 입니다.
    - 이 코드로 생성한 데이터로 데이터 제너레이터를 `[Simplified] data_pipeline.ipynb` 파일에 별도 정리할 예정입니다.

In [4]:
def main():
    # zip 파일의 경로를 저장하는 변수
    zip_path = "./quickdraw-doodle-recognition_simplified.zip"

    # 원본 데이터 디렉토리의 경로
    raw_path = "./data/train_simplified/"

    # 무작위로 섞인 원본 데이터를 저장할 디렉토리의 경로
    raw_shuffle_data_path = raw_path + "shuffle_raw_gzs/"

    # 데이터를 여러 부분으로 나누기 위한 변수
    divide_shuffles = 100

    # 처리된 파일의 로그를 저장하는 파일의 경로
    processed_files_log = "./data/processed_files.log"  # 로그 파일 경로

    # raw_path 디렉토리 생성
    if not os.path.exists(raw_path):
        os.makedirs(raw_path)

    # raw_shuffle_data_path 디렉토리 생성
    if not os.path.exists(raw_shuffle_data_path):
        os.makedirs(raw_shuffle_data_path)

    # 로그 파일에서 이미 처리된 파일 목록 읽기
    if os.path.exists(processed_files_log):
        with open(processed_files_log, "r") as log:
            processed_files = set(log.read().splitlines())
    else:
        processed_files = set()

    # 압축 파일 처리
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        file_list = [file for file in zip_ref.namelist() if file.endswith(".csv")]
        
        # tqdm을 사용하여 진행 상황 시각화
        pbar = tqdm(total=len(file_list))
        for y, file in enumerate(file_list):
            # 파일 경로에서 파일 이름만 추출
            csv_file = file.split("/")[-1]

            # 진행 상황 표시를 위한 설명 설정
            pbar.set_description(f"Processing {csv_file}")

            # 이미 처리되지 않은 파일인 경우에만 처리
            if csv_file not in processed_files:
                # 압축 파일에서 파일 추출
                zip_ref.extract(file, raw_path)

                # 파일 전처리 함수 호출
                preprocess_file(csv_file, raw_path, raw_shuffle_data_path, divide_shuffles, y, processed_files_log)

            # 진행 상황 업데이트
            pbar.update(1)

        # 진행 상황 표시 닫기
        pbar.close()

if __name__ == "__main__":
    main()


Processing giraffe.csv: 100%|██████████| 340/340 [5:37:47<00:00, 59.61s/it]


### Zip파일 만들기(팀원 공유용)

- 이 코드는 원본 데이터를 압축 파일로 변환하는 과정을 담당합니다. 
    - 먼저, 원본 데이터가 위치한 디렉토리(`raw_path`)와 무작위로 섞인 데이터를 저장할 디렉토리(`raw_shuffle_data_path`)의 경로를 설정합니다. 
    - 그 후, `raw_shuffle_data_path` 디렉토리 내의 모든 `.gz` 파일을 찾아 리스트(`gz_files`)에 저장합니다. 
    - 이 파일들은 `zipfile.ZipFile`을 사용하여 `zip_filename`에 지정된 경로의 압축 파일로 저장됩니다. 
    - 각 파일이 압축 파일에 추가될 때마다 `tqdm` 라이브러리를 사용하여 진행 상황을 시각적으로 표시합니다.

In [10]:
# 원본 데이터 디렉토리의 경로
raw_path = "./data/train_simplified/"

# 무작위로 섞인 원본 데이터를 저장할 디렉토리의 경로
raw_shuffle_data_path = raw_path + "shuffle_raw_gzs/"

# 압축 파일 경로를 지정
zip_filename = './train_simplified_chunked.zip'

# 폴더 내의 .gz 파일 목록을 가져옴
gz_files = [file for file in os.listdir(raw_shuffle_data_path) if file.endswith('.gz')]

# tqdm 진행 바를 설정
with tqdm(total=len(gz_files), unit="file") as pbar:
    pbar.set_description("압축 중")

    # zipfile 객체를 생성하여 파일을 하나씩 압축에 추가
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for file in gz_files:
            # 각 파일을 zip 파일에 추가
            zipf.write(os.path.join(raw_shuffle_data_path, file), arcname=file)
            # 진행 바 업데이트
            pbar.update(1)


압축 중: 100%|██████████| 100/100 [09:20<00:00,  5.60s/file]
